# Comparing G16 vs ML for the Referee

In [17]:
# imports
import numpy as np
import os, sys

from linetools.utils import dv_from_z

from dla_cnn.io import load_ml_dr7, load_ml_dr12, load_garnett16
from dla_cnn.catalogs import match_boss_catalogs

sys.path.append(os.path.abspath("../Analysis/py"))
import defs as analy_defs

## Setup

In [2]:
g16_pcut = analy_defs.g16_pcut
dztoler = 0.015
g16_pcut

0.95

## Load em up

### ML

In [3]:
# Load BOSS ML
_, dr12_abs = load_ml_dr12()
# Cut on DLA
dlas = dr12_abs['NHI'] >= 20.3
no_bals = dr12_abs['flg_BAL'] == 0
high_conf = dr12_abs['conf'] > 0.9
#
zem = (dr12_abs['zem'] > dr12_abs['zabs']) & (dr12_abs['zem'] > 2.15)  # G16 cut on zem
zcut = dr12_abs['zabs'] > 2.15
zprox = dr12_abs['zabs'] < (dr12_abs['zem'] - 3000./3e5)
# cut
dr12_cut = dlas & no_bals & high_conf & zem & zcut  & zprox
dr12_dla = dr12_abs[dr12_cut]
len(dr12_dla)

15922

### Load Garnett

In [4]:
g16_abs = load_garnett16()
g_dlas = g16_abs['log.NHI'] >= 20.3
g_conf = g16_abs['pDLAD'] >= g16_pcut
# Cut
g_cut = g_dlas & g_conf
g16_dlas = g16_abs[g_cut]
len(g16_dlas)

/home/xavier/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater_equal
  
/home/xavier/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater_equal
  This is separate from the ipykernel package so we can avoid doing imports until


18173

## Match ML to G16

In [5]:
dr12_to_g16 = match_boss_catalogs(dr12_dla, g16_dlas, dztoler=dztoler)
matched = dr12_to_g16 >= 0
g16_idx = dr12_to_g16[matched]
print("We matched {:d} of {:d} DLAs between high quality ML and G16 within dz={:g}".format(
    np.sum(matched), np.sum(dr12_cut), dztoler))

high_conf = (dr12_dla['conf'][matched] > 0.9) & (g16_dlas['pDLAD'][g16_idx] > analy_defs.g16_pcut)
print("Of these, {:d} are high confidence in both".format(np.sum(high_conf)))

We matched 10205 of 15922 DLAs between high quality ML and G16 within dz=0.015
Of these, 10205 are high confidence in both


### Consider large dNHI

In [6]:
NHI = dr12_dla['NHI'][matched]
dNHI = dr12_dla['NHI'][matched] - g16_dlas['log.NHI'][g16_idx]

In [7]:
large_dNHI = np.abs(dNHI) > 0.5
np.sum(large_dNHI)

133

### Examine an ML DLA not in G16

In [27]:
not_matched = dr12_to_g16 < 0
np.sum(not_matched)
#
not_idx = np.where(not_matched)[0]

In [29]:
jj = not_idx[0]
dr12_dla[jj]

Plate,Fiber,MJD,RA,DEC,zem,zabs,NHI,sigNHI,conf,flg_BAL,SNR
int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64
3587,617,55182,8.0754,0.1354,3.78847770165,2.95625578063,20.6791075122,0.0624429993331,0.9625,0,1.3497


#### Match to full G16 catalog

In [30]:
jj

1

In [33]:
jj1 = match_boss_catalogs(dr12_dla[jj:jj+2], g16_abs, dztoler=1.)  # Needed 2 for the code to run
jj1

array([ 9039, 11200])

In [34]:
g16_abs[9039]

THINGID,SDSS,Plate,MJD,Fiber,RAdeg,DEdeg,z_QSO,SNR,flg_BAL,zmin,zmax,log.pnDLA,log.pDLA,log.pDnDLA,log.pDDLA,pnDLAD,pDLAD,z_DLA,log.NHI
int64,str18,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
96897814,003218.09+000807.3,3587,55182,617,8.0753961,0.1353659,3.7994,1.3497,0,2.6096,3.7894,-0.10889,-2.2714,-1237.94,-1228.96,0.0010918,0.998908,2.8347,20.301


In [19]:
dv_from_z(2.956255, 2.8347)

<Quantity 9352.48726067834 km / s>

----

## Flip me

In [16]:
g16_to_dr12 = match_boss_catalogs(g16_dlas, dr12_dla, dztoler=dztoler, reverse=True)
matched2 = g16_to_dr12 >= 0
dr12_idx = g16_to_dr12[matched2]
print("We matched {:d} of {:d} DLAs between high quality ML and G16 within dz={:g}".format(
    np.sum(matched2), np.sum(g_cut), dztoler))

We matched 10205 of 18173 DLAs between high quality ML and G16 within dz=0.015


In [15]:
matched2

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)